# 마지막 교육세션 과제에 오신 여러분을 모두 환영합니다!

이번 과제의 목표는 인콘 데이터셋을 어떻게 접근할지에 대한 방향성을 잡아보는 것에 있습니다.

그렇다보니 별도의 정답값이 존재하지 않습니다.

코드보다는 어떤 파이프라인을 구성할지에 구상을 잡아두었는데요.

다음 질문들을 코드들을 채워 답해볼까요?

또한 이 데이터셋 자체로 확인하기 어렵다면 `어렵다..~`라고 편하게 작성해주시면 됩니다.

---
### 문제 제시 분해

---

문제 제시: 우리의 **성장세가 점차 둔화**되고 있다는 우려의 목소리가 나오고 있습니다.

질문: 성장세가 점차 둔화는 어떤 지표를 통해 파악할 수 있을까요?<br>
답:

In [2]:
# 지표를 구성해주세요.

# 금년 매출 성장률 / 전년 매출 성장률 : 이 수치가 1보다 작으면 성장 정체
# 금년 시장 점유율 / 전년 시장 점유율 : //
# 금년 순수 신규 고객 수 / 전년 순수 신규 고객 수 : //

문제 제시: `수익성을 극대화하고 재고 효율성을 개선`할 수 있는 전략적 아이디어를 제시하는 것입니다.

질문:

* `수익성`은 어떻게 정의하면 될까요?
* `재고 효율성`은 어떻게 정의되나요?

답:
- 수익성
  - 영업이익률: (영업이익/매출액) * 100
  - 순이익률: (당기순이익/매출액) * 100
  - 매출 총이익률: (매출 총이익/매출액) * 100

- 재고 효율성
  - 재고회전률: 매출원가 / 평균 재고 자산
  - 재고보유기간: 365 / 재고회전율

In [3]:
# 데이터셋을 불러와 확인해봅시다.
import pandas as pd
sales = pd.read_csv('drinksight-2024-sales.csv')
purchase = pd.read_csv('drinksight-2024-purchases.csv')
price = pd.read_csv('drinksight-2024-purchase-prices.csv')
invoice = pd.read_csv('drinksight-2024-vendor-invoice.csv')
beginv = pd.read_csv('drinksight-2024-begin-inventory.csv')
endinv = pd.read_csv('drinksight-2024-end-inventory.csv')


# sales_date 칼럼 날짜형으로 변환
sales['sales_date'] = pd.to_datetime(sales['sales_date'], format = '%Y-%m-%d')
# week 칼럼 만들기
sales['week'] = sales['sales_date'].dt.isocalendar().week
# day of week
sales['weekday'] = sales['sales_date'].dt.day_name()


# size 칼럼 단위 mL로 통일 및 숫자형으로 변환하는 함수 정의
def convert_size_to_ml(size):
    if not isinstance(size, str):
        return None

    size_lower = size.lower()

    try:
        if 'liter' == size_lower:
            return float(1000)
        elif '3/100ml' == size_lower:
            return float(300)
        elif '5/2 oz' == size_lower:
            return float(5 * 2 * 29.5735295625)
        elif '750ml + 1/' == size_lower:
            return float(750)
        elif '750ml + 2/' == size_lower:
            return float(750)
        elif '750ml + 3/' == size_lower:
            return float(750)
        elif size_lower.endswith('oz'):
            return float(size_lower.replace('oz', '').strip()) * 29.5735295625
        elif size_lower.endswith('gal'):
            return float(size_lower.replace('gal', '').strip()) * 3785.411784
        elif 'ml' in size_lower and 'pk' in size_lower:
            parts = size_lower.split()
            volume_per_unit = float(parts[0].replace('ml', '').strip())
            quantity = float(parts[1].strip())
            return volume_per_unit * quantity
        elif size_lower.endswith('ml'):
            return float(size_lower.replace('ml', '').strip())
        elif size_lower.endswith('l'):
            return float(size_lower.replace('liter', '').replace('l', '').strip()) * 1000
        else:
            return float(size_lower.strip())

    except ValueError:
        print(size_lower)
        return size

# datetime 자료형으로 변환
purchase['purchase_order_date'] = pd.to_datetime(purchase['purchase_order_date'], format = '%Y-%m-%d')
purchase['receiving_date'] = pd.to_datetime(purchase['receiving_date'], format = '%Y-%m-%d')
purchase['invoice_date'] = pd.to_datetime(purchase['invoice_date'], format = '%Y-%m-%d')
purchase['pay_date'] = pd.to_datetime(purchase['pay_date'], format = '%Y-%m-%d')

# size 칼럼 전처리
purchase['size'] = purchase['size'].apply(convert_size_to_ml)

질문: `수익성`은 어떤 자료의 척도를 지니나요?<br>
답: sales_df

질문: 위 답에 근거해 어떤 접근을 해야할지 고민해봅시다...회귀? 분류? 군집화?<br>
답: 회귀.....????

질문: 좋습니다. 그 방법을 선택하시기로 하셨군요! 그렇다면 이를 `수익성`을 설명하는 설명변수로 무엇을 선택하실건가요?<br>
답: 7월, 8월의 quantity 총합 계산

질문: 위 설명변수들이 선택된 이유가 별도로 있나요?<br>
답: 확인해 보니 8월에 매출이 급감했으니 전월이랑 비교할 필요가 있을 것 같음

In [4]:
# 좋습니다. 위 생각한대로 코드를 구성하고 모델을 평가해봅시다.
import pandas as pd
import numpy as np # 0으로 나누는 것을 방지하기 위해 import

sales['sales_date'] = pd.to_datetime(sales['sales_date'])

sales['month'] = sales['sales_date'].dt.strftime('%Y-%m')

sales_jul_aug = sales[sales['month'].isin(['2024-07', '2024-08'])]

#월별 총판매량
monthly_quantity_summary = sales_jul_aug.groupby('month')['sales_quantity'].sum()

print("--- 월별 총 판매량 비교 ---")
print(monthly_quantity_summary)
print("\\n" + "="*30 + "\\n")

#판매량 상세비교시작
item_sales_comparison = sales_jul_aug.groupby(['item', 'month'])['sales_quantity'].sum().unstack(fill_value=0)


item_sales_comparison['quantity_change'] = item_sales_comparison['2024-08'] - item_sales_comparison['2024-07']
item_sales_comparison['pct_change'] = (item_sales_comparison['quantity_change'] / item_sales_comparison['2024-07'].replace(0, np.nan)) * 100

top_fallers = item_sales_comparison.sort_values(by='quantity_change', ascending=True)

print("--- 7월 대비 8월 상품별 판매량 변화 (감소폭 큰 순) ---")
print(top_fallers.head(10))


--- 월별 총 판매량 비교 ---
month
2024-07    3439648
2024-08    2892266
Name: sales_quantity, dtype: int64
\n==============================\n
--- 7월 대비 8월 상품별 판매량 변화 (감소폭 큰 순) ---
month                         2024-07  2024-08  quantity_change  pct_change
item                                                                       
Grey Goose Vodka                65469    18118           -47351  -72.325834
Firefly Sweet Tea Vodka         24169     2908           -21261  -87.968058
Bacardi Superior Rum            34975    18253           -16722  -47.811294
Absolut 80 Proof                45887    31448           -14439  -31.466428
Kendall Jackson Chard Vt RSV    31320    18908           -12412  -39.629630
Ketel One Vodka                 24827    14001           -10826  -43.605752
Jack Daniels No 7 Black         37725    27044           -10681  -28.312790
Smirnoff Traveler               27307    17096           -10211  -37.393342
Malibu Rum                      23881    14470            -9411  -39

질문: `재고 효율성`은 어떤 자료의 척도를 지니나요?<br>
답: purchase, purchase-prices, vendor invoice

질문: 위 답에 근거해 어떤 접근을 해야할지 고민해봅시다...회귀? 분류? 군집화?<br>
답: 분류..??

질문: 좋습니다. 그 방법을 선택하시기로 하셨군요! 그렇다면 이를 `재고 효율성`을 설명하는 설명변수로 무엇을 선택하실건가요?<br>
답: 전체적인 발주 흐름 확인

질문: 위 설명변수들이 선택된 이유가 별도로 있나요?<br>
답: begin이랑 ending 데이터 상 brand_id 머지가 불가능함 > 상품별 재고 부족 여부 확인이 어렵

In [6]:
# 좋습니다. 재고 효율성도 한번 코드를 구성하고 모델을 평가해봅시다.
import pandas as pd

invoice['purchase_order_date'] = pd.to_datetime(invoice['purchase_order_date'])

invoice['month'] = invoice['purchase_order_date'].dt.strftime('%Y-%m')


monthly_purchase_summary = invoice.groupby('month').agg(
    total_purchase_qty=('quantity', 'sum'),
    purchase_order_count=('purchase_order_number', 'nunique')
).reset_index()


print("--- 월별 총 발주량 및 발주 횟수 ---")
print(monthly_purchase_summary)

--- 월별 총 발주량 및 발주 횟수 ---
      month  total_purchase_qty  purchase_order_count
0   2023-12              705759                   169
1   2024-01             2130443                   432
2   2024-02             2242110                   432
3   2024-03             2199278                   449
4   2024-04             2328996                   421
5   2024-05             3011592                   457
6   2024-06             3038355                   519
7   2024-07             3252027                   446
8   2024-08             3224148                   456
9   2024-09             2906400                   512
10  2024-10             3047143                   459
11  2024-11             2926435                   435
12  2024-12             2571691                   356


문제 제시: ~고객에게 `최고의 주류 경험을 제공`하는 것을 최우선 가치~

질문: 고객 기준에서 최고의 주류 경험을 느낀다는 것은 무엇일까요?<br>
*Hint: 당연한 이야기지만 최고의 주류 경험은 수치적으로 나타내기 쉽지 않습니다. 그렇기에 `Proxy Metrics`를 활용할 때가 온 것 같아요..~*<br>
답: 월별 판매량 확인해 보면 좋을 듯 or 재방문율?

In [5]:
# 확인해봅시다.

문제 제시: `운영 및 상품 관리 과정`에서 드러나는 문제 또는 개선 가능성

질문: Drink-Sight는 주류 전문 소매 체인입니다! 쉽게 말하면 술을 사와서 개인 소비자들에게 판매를 하고 있는 회사죠. 근데 이 각각의 물건들은 어떻게 소비자의 손까지 도달하게 되는걸까요?<br>
*hints: `vendor-invoice`, `purchases`, `purchase-prices`를 파악해 어떻게 가져오는지를 생각해봅시다*<br>
답: 도매업자들에게 구매하고, 그 가격에 따라 수익이 달라질 수 있음

문제 제시: `각기 다른 상권(City) 특성`을 가집니다.

질문: 각기 다른 상권 특성은 어떻게 확인할 수 있을까요? 도시에 대한 정보는 없었는데 말이죠?<br>
*hint: 하지만 이런 생각은 할 수 있을 것 같습니다. 우리나라 소주 지도를 아시나요? 각 지역별로 잘 나가는 소주가 다르다는 사실?*<br>
답: 주별로 잘 나가는 소주를 봐야 할듯

In [17]:
# 확인해봅시다.

---
### 지표 분해

---

문제를 분해하며 몇 가지 지표가 나왔나요?

아래 적어주시겠어요? 추가로 생각되는 지표가 있다면 더 적어보아도 됩니다!

*
*
*
*
*

`수익성`지표를 분해하겠습니다.

월별로, 기업별로 등등 분해해봅시다.<br>
특이사항을 보이는 부분이 있을까요?

In [18]:
# 확인해봅시다.

특이사항이 있었다면 그 원인은 무엇일까요?

positive한 특이사항이라면 그 feature를 강화하는 방향으로<br>
negative한 특이사항이라면 그 feature를 약화하는 방식으로<br>
기업의 운영을 더 개선할 수 있을 것 같습니다.

한번 작성해봅시다.

그 다음 지표도 분해해봅시다.

In [19]:
# 확인해봅시다.

특이사항이 있었다면 그 원인은 무엇일까요?

positive한 특이사항이라면 그 feature를 강화하는 방향으로<br>
negative한 특이사항이라면 그 feature를 약화하는 방식으로<br>
기업의 운영을 더 개선할 수 있을 것 같습니다.

한번 작성해봅시다.

3번째 지표도 분해해봅시다.

In [20]:
# 확인해봅시다.

특이사항이 있었다면 그 원인은 무엇일까요?

positive한 특이사항이라면 그 feature를 강화하는 방향으로<br>
negative한 특이사항이라면 그 feature를 약화하는 방식으로<br>
기업의 운영을 더 개선할 수 있을 것 같습니다.

한번 작성해봅시다.

---
### 가설 설정

---

지표들을 해석하며 여러가지 positive혹은 negative한 관계들을 포착할 수 있었을텐데요.

그 이유는 무엇일까요?

한번 생각해봅시다.

또한 통계적 유의성도 고민해봐야겠는데요?

In [ ]:
# 파악하고 작성해봅시다.

---
### 해결방안 제시

---

유의미한 가설관계가 설정하여 파악 완료하였다면 이에 미래를 바라볼 차례입니다

어떤 전략을 제안해야 Drink-Sight가 더 개선될 수 있을까요?

* ~~~
* ~~~
* ~~~

# 여기까집니다! 위 자료들 기초하여 인콘까지 화이팅하시길 바랍니다!